In [1]:
import pandas as pd
import numpy as np
import csv
import os
import math

In [2]:
from pathlib import Path
import sys
import os

# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fremontdropbox import get_dropbox_location

dropbox_dir = get_dropbox_location()

#for 2013, 2017, 2019 raw ADT
ADT_dir = dropbox_dir + '/Private Structured data collection/Data processing/Raw/Demand/Flow_speed/City'


PeMs_dir = dropbox_dir + '/Private Structured data collection/Data processing/Auxiliary files/Demand/Flow_speed/PeMs'
local_download =  str(os.path.join(Path.home(), "Downloads"))
re_formated_Processed_dir = dropbox_dir + '/Private Structured data collection/Data processing/Temporary exports to be copied to processed data/Flow_processed'


#for processed: 
Processed_dir = dropbox_dir + '/Private Structured data collection/Data processing/Auxiliary files/Demand/Flow_speed/Flow_processed'



In [4]:
input_files_excel = os.listdir(Processed_dir)
processed_files = []
for file in input_files_excel:
    if os.path.isdir(Processed_dir + "/" + file):
        processed_files.append(file)
        print(file)
        year = file.split(" ")[0]
processed_files.sort()            
print(processed_files)

2017 processed
2019 processed
2013 processed
2015 processed
['2013 processed', '2015 processed', '2017 processed', '2019 processed']


In [27]:
curr_year_dir + "/" + file_name

'/Users/LiJiayi/Fremont Dropbox/Theophile Cabannes/Private Structured data collection/Data processing/Auxiliary files/Demand/Flow_speed/Flow_processed/2015 processed/Washington Blvd betw. Paseo Padre and Mission.csv'

In [46]:
input_file = pd.read_csv(curr_year_dir + "/" + file_name)

In [5]:
to_be_concatenated = []
for year_processed in processed_files:
    curr_year_dir = Processed_dir + "/" + year_processed
    file_names = os.listdir(curr_year_dir)
    file_names.sort()
    for file_name in file_names:
        if file_name.split(".")[-1] == 'csv':
            input_file = pd.read_csv(curr_year_dir + "/" + file_name)
            file_day_one = input_file['Date'].apply(lambda x: x.split(" ")[0])[0]
            file_year = file_day_one.split("-")[0]
            if (file_year == '2017' or file_year == '2019') and (('WB' in file_name) or ('EB' in file_name) or ('NB' in file_name) or ('SB' in file_name)):
                file_direction = file_name.split(".")[0].split(" ")[-1]
                one_direction = input_file.transpose().iloc[[1], :]
                flow_information = one_direction
                flow_information['Id'] = 1
                flow_information['Direction'] = file_direction#two_directions.index.tolist()
            else:
                two_directions = input_file.transpose().iloc[[2, 3], :]
                flow_information = two_directions
                flow_information['Id'] = [0, 1]
                flow_information['Direction'] = two_directions.index.tolist()
                if file_name == 'WARM SPRINGS BLVD BT BROWN RD AND WARREN AVE  .csv':
                    print(flow_information)
            flow_information['year'] = file_year
            flow_information['Name'] = file_name
            flow_information['Day 1'] = file_day_one
            to_be_concatenated.append(flow_information)

In [6]:
df_total = pd.concat(to_be_concatenated, ignore_index=True)

ADT_dir = dropbox_dir + '/Private Structured data collection/Data processing/Raw/Demand/Flow_speed/ADT'
PeMs_dir = dropbox_dir + '/Private Structured data collection/Data processing/Auxiliary files/Demand/Flow_speed/PeMs'
local_download =  str(os.path.join(Path.home(), "Downloads"))
re_formated_Processed_dir = dropbox_dir + '/Private Structured data collection/Data processing/Temporary exports to be copied to processed data/Flow_processed'

curr_file = open(re_formated_Processed_dir + "/" + "Flow_processed_tmp.csv", "r", encoding= 'unicode_escape')#encoding='utf-8-sig').strip()
w = open(re_formated_Processed_dir + "/" +'Flow_processed_city.csv', 'w')
w2 = open(re_formated_Processed_dir + "/" +'Flow_processed_PeMS.csv', 'w')

legend = []
for k in range(3):
    for i in range(24):
        for j in range(4):
            legend = legend + ["Day " + str(k + 1) + " - " + str(i) + ":" + str(15 * j)]
            
City_legend = legend + ['Id', 'Direction', 'year', 'Name', 'Day 1']
df_total.columns = City_legend
df_total.dropna(subset=legend, how = 'all', inplace=True)
df_total.reset_index(drop=True, inplace=True)
df_total['Id'] = df_total.index

cols = df_total.columns.tolist()
cols = cols[-5:] + cols[0:-5]
df_total = df_total[cols]

In [7]:
df_total.to_csv('/Users/LiJiayi/Documents/Flow_processed_city.csv')

In [40]:
curr_file = open(re_formated_Processed_dir + "/" + "Flow_processed_tmp.csv", "r", encoding= 'unicode_escape')
ids = {}
curr_year = 0
curr_year_ids = []

for line in curr_file:
    if line.split(",")[1][0:4] == "PeMS":
        ids["PeMS"] = []
        continue
    if line.split(",")[1].split(".")[-1][0] == '4':
        ids["PeMS"].append(line.split(",")[0])
        continue
    new_year = line.split("/")[-1].split(" ")[0]
    if new_year == '2013' or new_year == '2015' or new_year == '2017' or new_year == '2019':
        if curr_year != 0:
            ids[curr_year] = curr_year_ids
        curr_year = new_year
        curr_year_ids = []
    else: 
        curr_id = line.split(",")[0]
        if curr_id != '':
            curr_year_ids.append(curr_id)
if curr_year != 0:
    ids[curr_year] = curr_year_ids
    

In [43]:
len(ids['2015'] + ids['2013'] + ids['2017'] + ids['2019'])

132

In [18]:
# reordering
df_total

['Day 1 - 0:0', 'Day 1 - 0:15', 'Day 1 - 0:30', 'Day 1 - 0:45', 'Day 1 - 1:0', 'Day 1 - 1:15', 'Day 1 - 1:30', 'Day 1 - 1:45', 'Day 1 - 2:0', 'Day 1 - 2:15', 'Day 1 - 2:30', 'Day 1 - 2:45', 'Day 1 - 3:0', 'Day 1 - 3:15', 'Day 1 - 3:30', 'Day 1 - 3:45', 'Day 1 - 4:0', 'Day 1 - 4:15', 'Day 1 - 4:30', 'Day 1 - 4:45', 'Day 1 - 5:0', 'Day 1 - 5:15', 'Day 1 - 5:30', 'Day 1 - 5:45', 'Day 1 - 6:0', 'Day 1 - 6:15', 'Day 1 - 6:30', 'Day 1 - 6:45', 'Day 1 - 7:0', 'Day 1 - 7:15', 'Day 1 - 7:30', 'Day 1 - 7:45', 'Day 1 - 8:0', 'Day 1 - 8:15', 'Day 1 - 8:30', 'Day 1 - 8:45', 'Day 1 - 9:0', 'Day 1 - 9:15', 'Day 1 - 9:30', 'Day 1 - 9:45', 'Day 1 - 10:0', 'Day 1 - 10:15', 'Day 1 - 10:30', 'Day 1 - 10:45', 'Day 1 - 11:0', 'Day 1 - 11:15', 'Day 1 - 11:30', 'Day 1 - 11:45', 'Day 1 - 12:0', 'Day 1 - 12:15', 'Day 1 - 12:30', 'Day 1 - 12:45', 'Day 1 - 13:0', 'Day 1 - 13:15', 'Day 1 - 13:30', 'Day 1 - 13:45', 'Day 1 - 14:0', 'Day 1 - 14:15', 'Day 1 - 14:30', 'Day 1 - 14:45', 'Day 1 - 15:0', 'Day 1 - 15:15', '

,year,Name,Id,Day 1,Direction,Day 1 - 0:0,Day 1 - 0:15,Day 1 - 0:30,Day 1 - 0:45,Day 1 - 1:0,...,Day 3 - 21:30,Day 3 - 21:45,Day 3 - 22:0,Day 3 - 22:15,Day 3 - 22:30,Day 3 - 22:45,Day 3 - 23:0,Day 3 - 23:15,Day 3 - 23:30,Day 3 - 23:45
0,0,NB,0,PASEO PADRE PKWY BT CAMINO DEL CAMPO AND PINE ...,2017-05-16,2,2,2,3,0,...,20,11,13,9,9,11,3,5,6,4
1,1,SB,1,PASEO PADRE PKWY BT CAMINO DEL CAMPO AND PINE ...,2017-05-16,2,0,1,1,0,...,21,15,9,4,7,4,2,4,3,2
2,0,WB,2,GRIMMER BLVD BT PASEO PADRE PKWY AND OSGOOD RD...,2017-05-16,2,1,1,1,2,...,27,9,18,5,8,8,3,4,5,3
3,1,EB,3,GRIMMER BLVD BT PASEO PADRE PKWY AND OSGOOD RD...,2017-05-16,5,11,3,3,2,...,46,47,28,22,27,19,20,15,8,5
4,0,WB,4,WARREN AVE BT CURTNER RD AND WARM SPRINGS BLVD...,2017-05-16,0,0,1,1,2,...,11,11,13,9,7,7,12,4,5,2
5,1,EB,5,WARREN AVE BT CURTNER RD AND WARM SPRINGS BLVD...,2017-05-16,2,0,2,1,1,...,6,8,9,4,0,3,2,1,0,3
6,0,NB,6,MISSION BLVD S OF WASHINGTON BLVD SIGNAL.csv,2017-05-16,6,6,6,5,3,...,46,53,31,24,23,26,18,19,15,19
7,1,SB,7,MISSION BLVD S OF WASHINGTON BLVD SIGNAL.csv,2017-05-16,5,7,5,5,6,...,29,26,28,20,18,19,11,13,9,15
8,0,WB,8,DRISCOLL RD BT COMAC TERRACE AND DENISE ST.csv,2017-05-16,9,8,6,4,1,...,37,34,22,17,16,15,12,15,12,0
9,1,EB,9,DRISCOLL RD BT COMAC TERRACE AND DENISE ST.csv,2017-05-16,12,10,7,6,3,...,65,59,39,42,29,39,32,19,18,17


,Day 1 - 0:0,Day 1 - 0:15,Day 1 - 0:30,Day 1 - 0:45,Day 1 - 1:0,Day 1 - 1:15,Day 1 - 1:30,Day 1 - 1:45,Day 1 - 2:0,Day 1 - 2:15,...,Day 3 - 22:45,Day 3 - 23:0,Day 3 - 23:15,Day 3 - 23:30,Day 3 - 23:45,year,Name,Id,Day 1,Direction
0,2,2,2,3,0,1,2,2,0,0,...,11,3,5,6,4,0,NB,0,PASEO PADRE PKWY BT CAMINO DEL CAMPO AND PINE ...,2017-05-16
1,2,0,1,1,0,0,2,0,1,0,...,4,2,4,3,2,1,SB,1,PASEO PADRE PKWY BT CAMINO DEL CAMPO AND PINE ...,2017-05-16
2,2,1,1,1,2,0,1,0,0,0,...,8,3,4,5,3,0,WB,2,GRIMMER BLVD BT PASEO PADRE PKWY AND OSGOOD RD...,2017-05-16
3,5,11,3,3,2,3,1,2,1,2,...,19,20,15,8,5,1,EB,3,GRIMMER BLVD BT PASEO PADRE PKWY AND OSGOOD RD...,2017-05-16
4,0,0,1,1,2,0,0,0,1,0,...,7,12,4,5,2,0,WB,4,WARREN AVE BT CURTNER RD AND WARM SPRINGS BLVD...,2017-05-16
5,2,0,2,1,1,0,1,0,0,0,...,3,2,1,0,3,1,EB,5,WARREN AVE BT CURTNER RD AND WARM SPRINGS BLVD...,2017-05-16
6,6,6,6,5,3,4,3,0,4,4,...,26,18,19,15,19,0,NB,6,MISSION BLVD S OF WASHINGTON BLVD SIGNAL.csv,2017-05-16
7,5,7,5,5,6,3,4,2,0,3,...,19,11,13,9,15,1,SB,7,MISSION BLVD S OF WASHINGTON BLVD SIGNAL.csv,2017-05-16
8,9,8,6,4,1,1,1,2,4,1,...,15,12,15,12,0,0,WB,8,DRISCOLL RD BT COMAC TERRACE AND DENISE ST.csv,2017-05-16
9,12,10,7,6,3,3,2,3,1,6,...,39,32,19,18,17,1,EB,9,DRISCOLL RD BT COMAC TERRACE AND DENISE ST.csv,2017-05-16


In [2]:
# load excel

df = pd.DataFrame()

for f in os.listdir():
    if('xls' in f):
        input_file = pd.read_excel(f).head(289)
        input_file = input_file.transpose().reset_index().rename(columns={'index':'id', 0:'Direction'}).head(5)
        input_file['id']=''
        input_file['Direction']=['', 'NB', 'SB', 'EB', 'WB']
        if math.isnan(input_file.iloc[1,2]) and math.isnan(input_file.iloc[2,2]):
            # if northbound and southbound are NAN
            input_file = input_file.drop([1,2])

        else:
            input_file = input_file.drop([3,4])
        input_file.iloc[0,2].date()
        input_file.insert(2, 'Day 1',input_file.iloc[0,2].date() )
        input_file.insert(0, 'Name', f)
        input_file = input_file.drop(0)
        df = df.append(input_file)

# input_file = pd.read_excel('Durham Rd betw. I-680 and Mission.xls').head(289)

# input_file = input_file.transpose().reset_index().rename(columns={'index':'id', 0:'Direction'}).head(5)
# input_file['id']=''
# input_file['Direction']=['', 'NB', 'SB', 'EB', 'WB']

# if math.isnan(input_file.iloc[1,2]) and math.isnan(input_file.iloc[2,2]):
#     # if northbound and southbound are NAN
#     input_file = input_file.drop([1,2])
    
# else:
#     input_file = input_file.drop([3,4])

# input_file.iloc[0,2].date()
# input_file.insert(2, 'Day 1',input_file.iloc[0,2].date() )
# #input_file = input_file.drop(0)
# # input_file.insert(2, "Age", [21, 23, 24, 21], True) 
# input_file.append(input_file)
# df = df.append(input_file)
df = df.reset_index().drop('index', axis=1)
df

,Name,id,Direction,Day 1,1,2,3,4,5,6,...,279,280,281,282,283,284,285,286,287,288
0,Washington Blvd betw. Driscoll and Paseo Padre...,,EB,2015-03-17,6,5,4,5,7,4,...,76,76,55,47,19,20,23,34,16,10
1,Washington Blvd betw. Driscoll and Paseo Padre...,,WB,2015-03-17,12,7,8,2,4,3,...,47,67,41,35,25,17,12,18,13,11
2,Paseo Padre Pkwy betw. Mission and Curtner.xls,,EB,2015-03-31,2,1,2,1,2,2,...,27,19,10,13,9,9,9,9,3,6
3,Paseo Padre Pkwy betw. Mission and Curtner.xls,,WB,2015-03-31,3,0,0,1,1,0,...,11,10,11,10,6,6,12,13,0,4
4,Warren Ave betw. Curtner and Warm Springs.xls,,EB,2015-04-21,4,3,2,0,1,0,...,11,15,19,13,11,4,10,8,4,3
5,Warren Ave betw. Curtner and Warm Springs.xls,,WB,2015-04-21,1,2,0,0,0,0,...,11,6,6,5,3,2,5,1,2,2
6,Mission Blvd betw. Durham and Curtner.xls,,NB,2015-03-17,1,4,8,4,5,3,...,57,44,45,42,23,29,24,23,18,13
7,Mission Blvd betw. Durham and Curtner.xls,,SB,2015-03-17,4,6,2,3,0,2,...,25,41,30,16,7,16,11,7,5,3
8,Mission Blvd betw. St. Josephs and Pine.xls,,NB,2015-03-10,7,4,3,4,6,2,...,80,33,37,63,22,32,10,6,11,7
9,Mission Blvd betw. St. Josephs and Pine.xls,,SB,2015-03-10,2,5,4,4,3,2,...,38,24,20,21,22,8,12,6,4,6


In [86]:
df.to_csv('2015_Flow_Processed.csv')